# Validation sampling code for Forrest local SDGs systems model

In [1]:
# set working directory locations
import sys
sys.path.append(r'C:\temp')
sys.path.append(r'C:\Users\szeteyka\OneDrive - Deakin University\PhD\Models\Moallemi_et_al_SDG_SSP_Assessment-main')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import load_results, ema_logging
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import morris
import os
from datetime import datetime

C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\em_framework\evaluators.py:21: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn(
C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:48: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)


In [3]:
if __name__ == "__main__":
    ema_logging.log_to_stderr(ema_logging.INFO)

from ema_workbench import (TimeSeriesOutcome, 
                                   perform_experiments,
                                   RealParameter, 
                                   CategoricalParameter,
                                   Constant,
                                   ema_logging, 
                                   save_results,
                                   load_results)

from ema_workbench.connectors.vensim import VensimModel
directory = 'C:/temp/model/'
model = VensimModel("forrestvalidation", wd=directory, model_file=r'C:/temp/model/LocalSDGsmodel.vpmx')        

#define file and sheet for outcomes
df_out = pd.read_excel(directory+'SensitivityVariables_validation.xlsx', sheet_name='outcomes')

#define column headers for outcomes
model.outcomes = [TimeSeriesOutcome(out) for out in df_out['Outcome']]

#define file and sheet for uncertainties 
df_unc = pd.read_excel(directory+'SensitivityVariables_validation.xlsx', sheet_name='uncertainties')

#define column headers for lower and upper bounds of uncertainties
model.uncertainties = [RealParameter(row['Variable'], row['Lower'], row['Upper']) for index, row in df_unc.iterrows()]

from ema_workbench import MultiprocessingEvaluator
from ema_workbench.em_framework.evaluators import (MC, LHS, FAST, FF, PFF, SOBOL, MORRIS)

import time
start = time.time()

nr_experiments = 20000
with MultiprocessingEvaluator(model) as evaluator:
    results = perform_experiments(model, nr_experiments,
                                      evaluator=evaluator)

end = time.time()
print("took {} seconds".format(end-start))    

#define results location and filename
fn = 'C:/temp/results/20000exp.bz2'

# If scenario results file exists, move to backup (replacing the backup if it exists)
    
if os.path.isfile(fn):
    os.replace(fn, fn + '.bak')

save_results(results, fn)

[MainProcess/INFO] using 64 bit vensim
C:\Users\szeteyka\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
[MainProcess/INFO] pool started
[MainProcess/INFO] performing 20000 scenarios * 1 policies * 1 model(s) = 20000 experiments
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] 4000 cases completed
[MainProcess/INFO] 6000 cases completed
[MainProcess/INFO] 8000 cases completed
[MainProcess/INFO] 10000 cases completed
[MainProcess/INFO] 12000 cases completed
[MainProcess/INFO] 14000 cases completed
[MainProcess/INFO] 16000 cases completed
[MainProcess/INFO] 18000 cases completed
[MainProcess/INFO] 20000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


took 2095.486225605011 seconds


[MainProcess/INFO] results saved successfully to C:\temp\results\20000exp.bz2


In [7]:
#load results from previously defined save location
results = load_results('C:/temp/results/20000exp.bz2')
experiments, outcomes = results
dict_outcomes = outcomes

[MainProcess/INFO] results loaded succesfully from C:\temp\results\20000exp.bz2


In [8]:
#generate dataframe of outcome results

outcomes_df = pd.DataFrame(columns=list(dict_outcomes.keys()))
for key in set(list(dict_outcomes.keys())): 
    outcomes_df[key] = dict_outcomes[key][:, 0:].ravel() # Specify from when to plot the data 
outcomes_dict = outcomes_df

outcomes_df

,TIME,Young,Mid,Old,total population,Fertilisation Intensity,fertiliser consumption,Agriculture Land,Forest Land,Housing Land,...,Total Tourism Productivity,labour input tourism,Total Other Productivity,labour input other,profitability increase from regenerative agriculture,FFDI model,rainfall linear model,temperature linear model,CO2 linear model,road accidents
0,2000.00,45.000000,106.00000,20.000000,171.00000,0.254000,901.57306,3549.5000,4436.0000,57.30000,...,2.500000e+04,25.136100,1175000.0,19.775360,1.799999,1.500301,760.64996,16.939999,362.56003,9.600000
1,2000.25,44.432674,106.43703,20.202385,171.07208,0.252412,891.61163,3532.3591,4382.5625,75.49090,...,2.500000e+04,25.239733,1186802.6,19.856892,1.791499,1.506956,759.97220,16.948101,363.00253,9.597594
2,2000.50,43.854996,106.87710,20.395897,171.12799,0.250835,882.00400,3516.2727,4333.2324,89.15084,...,2.500000e+04,25.344090,1201680.4,19.938992,1.782999,1.513611,759.29450,16.956203,363.44500,9.595188
3,2000.75,43.269512,107.31937,20.580843,171.16972,0.249267,872.59420,3500.6375,4286.2340,100.61086,...,2.507239e+04,25.448967,1219562.5,20.021502,1.774499,1.520266,758.61670,16.964306,363.88750,9.592783
4,2001.00,42.661636,107.77205,20.758183,171.19186,0.247709,863.32830,3485.2480,4240.9326,110.68280,...,2.529022e+04,25.556310,1240458.9,20.105953,1.765999,1.526921,757.93896,16.972408,364.33002,9.590377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019995,2049.00,22.227068,203.07892,48.130300,273.43628,0.074328,150.62883,2026.5374,1337.8948,644.92670,...,9.905995e+06,48.156715,4443870.5,37.886402,0.133999,2.804681,627.81100,18.528088,449.29000,9.134113
4019996,2049.25,22.305939,203.85303,48.339996,274.49896,0.073864,149.29971,2021.2887,1330.6632,646.06360,...,9.977092e+06,48.340286,4466085.5,38.030823,0.125499,2.811336,627.13324,18.536190,449.73250,9.131765
4019997,2049.50,22.380125,204.64080,48.550808,275.57172,0.073402,147.98560,2016.0981,1323.4408,647.19460,...,1.004839e+07,48.527090,4488340.0,38.177788,0.116999,2.817991,626.45544,18.544292,450.17502,9.129417
4019998,2049.75,22.460320,205.42094,48.762800,276.64404,0.072943,146.68626,2010.9652,1316.2281,648.31964,...,1.011988e+07,48.712086,4510543.5,38.323330,0.108499,2.824646,625.77770,18.552395,450.61752,9.127070


In [1]:
#print outcomes to csv file

# datetime object containing current date and time
now = datetime.now()
 
#format as YYmmdd-HHMM
dt_string = now.strftime("%Y%m%d-%H%M")

#define save file location for results and export to csv
outcomes_df.to_csv('C:/temp/results/{}_validation_outcomes.csv'.format(dt_string))

NameError: name 'datetime' is not defined